<a href="https://colab.research.google.com/github/luckycontrol/DeepLearning_tensorflow/blob/main/04_%EC%86%90%EC%8B%A4%ED%95%A8%EC%88%98(Loss_Function).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

손실함수란 신경망이 얼마나 맞추지 못했냐 라는 지표이다.
- 즉 낮을수록 좋은 값.
- 손실함수에 영향을 미치는 것은 가중치(W)와 편향(B)
- 신경망의 학습이란 손실함수의 값이 낮은 가중치와 편향을 구하는 것.

# MSE
 평균 제곱 오차

In [ ]:
import numpy as np

# 모델이 2로 예측 했을 확률이 0.6
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

# 실제 정답은 2이다.
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
def mean_squard_error(y, t):
  return 0.5 * np.sum((y - t) ** 2)

In [ ]:
print("MSE: {}".format(mean_squard_error(np.array(y), np.array(t))))

MSE: 0.09750000000000003


In [ ]:
# 모델이 7로 예측할 확률이 0.6
y_error = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

In [ ]:
print("MSE: {}".format(mean_squard_error(np.array(y_error), np.array(t))))

MSE: 0.5975


# CEE ( Cross Entropy Error )
- 교차 엔트로피 에러

In [ ]:
# 로그 공부.

def cross_entropy_error(y, t):
  delta = 1e-7 # 아주 작은 값
  return -np.sum(t * np.log(y + delta)) # 델타를 더해주는 이유 : 로그가 0이되면 무한대를 의미. 무한대로 발산하지 않도록 만들어주기위해 아주 작은 값을 더해준다.

In [ ]:
# 모델이 2로 예측 했을 확률이 0.6
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

# 실제 정답은 2이다.
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 모델이 7로 예측할 확률이 0.6
y_error = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]

print("y의 CES: {:.3f}".format(cross_entropy_error(np.array(y), np.array(t))))
print("y의 CES: {:.3f}".format(cross_entropy_error(np.array(y_error), np.array(t))))

y의 CES: 0.511
y의 CES: 2.303


# 미니배치

In [ ]:
from tensorflow.keras import datasets

mnist = datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train[0].shape

(28, 28)

In [ ]:
y_train.shape

(60000,)

y_train은 OneHotEncoding이 필요.

In [ ]:
# step 1 : (60000, 1) 만들기
y_train_reshape = y_train.reshape(y_train.shape[0], -1)

In [ ]:
y_train_reshape.shape

(60000, 1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

y_train_dummy = OneHotEncoder().fit_transform(y_train_reshape)
y_train_dummy = y_train_dummy.toarray()

y_train_dummy.shape

(60000, 10)

In [ ]:
x_train_dummy = x_train.reshape(x_train.shape[0], -1)
x_train_dummy.shape

(60000, 784)

In [ ]:
x_train_dummy.shape, y_train_dummy.shape

((60000, 784), (60000, 10))

미니배치 - 랜덤하게 뽑은 인덱스로 배치를 만든 것

In [ ]:
# 60000개의 데이터 중 무작위로 10개만 뽑아서 미니배치를 만들기.
# choice(범위 숫자, 개수)

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # 60000개 중 10개를 무작위로 뽑는다.

print(batch_mask)


[ 5866 53148 38041 53482 22886 53345 58932 43768  5347  8781]


In [ ]:
# 마스트에의해 선택된 데이터 뽑기
x_batch = x_train[batch_mask]
t_batch = y_train_dummy[batch_mask]

배치용 크로스 엔트로피 구하기 1

In [ ]:
def cross_entropy_error_v1(y, t):

  if y.ndim == 1: # batch가 한개가 들어와도 처리해주기 위해서.
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]

  # 배치마다의 오차를 모두 더해서 배치 크기로 나눠준다.
  return -np.sum(t * np.log(y)) / batch_size # OneHotEncoding된 레이블에만 사용 가능.

In [ ]:
# sparse_categorical_crossentropy
def cross_entropy_error_v2(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]

  return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size